# Работа с категориальными данными
Часто возникает задача разбить данные на категории не по количеству, а по качеству. Такая качественная информация представляется как принадлежность наблюдения отдельной категории, такой как пол, цвет или марка автомобиля и т.д. Однако не все категориальные данные одинаковы. Наборы категорий без внутреннего упорядочивания называются *номинальными*. Примеры номнальных категорий включают:
* синий, ркасный, зеленый;  
* мужчинв, женщина;  
* банан, клубника, яблоко.  

С другой стороны, когда набор категорий имеет определенное естественное упорядочивание, оно называется *порядковым*. Напрмер:
* низкий, средний, высокий;  
* молодой, старый;  
* согласен, нейтрален, не согласен.  

Более того категориальная информаия часто представлена в данных в виде вектора или столбцасимвольных значений. Например: `['Мэн', 'Техас', 'Делавэр']`. Проблема в том, что большинство алгоритмов машинного обучения требуют ввода числовых значений.

Алгоритм $k$ ближайших соседей предоставляет простой пример. ОДним из шагов в алгоритме является вычисление расстояний между наблюдениями – часто с использование евклидова расстояния:
$$\sqrt{\sum^{n}_{i=1}{(x_i - y_i)^2}}$$
где $x$ и $y$ – это два наблюдения, а $i$ – номер признака наблюдения.

Однако вычисление расстояния, очевидно, невозможно, если значение $x_i$ является строкой (например, `'Техас'`). Для того, чтобы его можно было ввести в уравнение евклидова расстояния, необходимо преобразовать это строковое значение в числовой формат. Важно сделать такое преобразование таким образом, чтобы оно правильно передавало информацию в категориях (упорядоченность, относительные интервалы между категориями и т.д.

***


### 1. Кодирование номинальных категориальных признаков
##### Задача
Дан признак с номинальными классами, который не имеет внутренней упорядоченности. Преобразовать признак в кодировку с одним активными состоянием.
##### Решение
Используем класс `LabelBinarizer` библиотеки `scikit-learn`

In [3]:
# Импортировать библиотеки
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

# Создать признак
feature = np.array([
    ['Texas'],
    ['California'],
    ['Texas'],
    ['Delaware'],
    ['Minesota'],
    ['Texas']
])

# Создать кодировщик одного активного состояния
one_hot = LabelBinarizer()

# Преобразовать признак в кодировку с одним активным состоянием
one_hot.fit_transform(feature)

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

Для вывода классов можно воспользоваться атрибутом `classes_`:

In [4]:
# Взглянуть на классы признака
one_hot.classes_

array(['California', 'Delaware', 'Minesota', 'Texas'], dtype='<U10')

Если требуется обратить кодирование с одним активным состоянием, необходимо применить метод `inverse_transform()`:

In [6]:
# Обратить кодирование с одним активным состоянием
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Minesota', 'Texas'],
      dtype='<U10')

Также для преобразования признака в кодировку с одним активным состоянием можно использовать библиотеку `pandas`:

In [7]:
# Импортировать библиотеку
import pandas as pd 

# Создать фиктивные переменные признака
pd.get_dummies(feature[:,0])

/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


,California,Delaware,Minesota,Texas
0,0,0,0,1
1,1,0,0,0
2,0,0,0,1
3,0,1,0,0
4,0,0,1,0
5,0,0,0,1


Другой полезной возможностью библиотеки `scikit-learn` является обработка ситуации, когда в каждом наблюдении перечисляется несколько классов:

In [8]:
# Создать мультиклассовый признак
multiclass_feature = [
    ('Texas', 'Florida'),
    ('California', 'Alabama'),
    ('Texas', 'Florida'),
    ('Delaware', 'Florida'),
    ('Texas', 'Alabama')
]

# Создать мультиклассовый кодировщик, преобразующий признак в кодировку с одним активным состоянием
one_hot_multiclass = MultiLabelBinarizer()

# Кодировать мультиклассовый признак в кодировку с одним активным состоянием
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [9]:
# Взглянуть на классы
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delaware', 'Florida', 'Texas'],
      dtype=object)

*Кодирование с одним активным сотоянием* (one-hot encoding) пришло из терминологии цифровых интегральных микросхем, где оно описывает конфигурацию микросхемы, в которой допускается, чтобы только один бит был положительным.

Может показаться, что более правильной стратегией будет та, в которой каждому классу будет назначено числовле значение, нпример: `Texas = 1`, `'California' = 2` и т.д. Однако, кодга классы не имеют  внутренней упорядоченности, числовые значения ошибочно задают порядок, которого нет.

Грамотной стратегией является создание в исходном признаке бинарного признака для класса, т.е. *кодирование с одним активным состоянием* или *фиктивизация*. Признаком в примере выше был вектор, содержащий четыре класса `'California'`, `'Delaware'`, `'Minesota'` и `'Texas'`. В кодировании с одним активным состояниемкаждый класс становится одноэлементным признаком с единицами, кодга класс появляется и нулями, кодга отсутствует.

Поскольку признак `feature` имел четыре класса, кодирование с одним активным состоянием вернул четыре бинарных признака, по одному для каждого класса. Используя кодирование с одним активным состоянием, можно фиксировать принадлежность наблюдения к классу, сохраняя при этом сведения о том, что в класса отстутствует какая-либо иерархия.

Следует отметить, что после кодирования признака в кодировку с одним активным состоянием рекомендуется отбрасывать один из закодированных в результирующей матрице признаков, чтобы избежать линейной зависимости.

##### Дополнительные материалы
* Статья "Ловушка фиктивной переменной": https://www.algosome.com/articles/dummy-variable-trap-regression.html    
* Статья "Исключение одного из столбцов при использовании кодирований с одним активным сотоянием": https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding  

### 2. Кодирование порядковых категориальных признаков
##### Задача
Дан порядковый ктегориальный признак. Выполнить его кодировку
##### Решение
Используем метод `repalce()` фрейма данных библиотеки `pandas` для преобразования строковых меток в числовые

In [11]:
# Создать признаки
df = pd.DataFrame(
    {'оценка': ['низкая', 'низкая', 'средняя', 'высокая', 'средняя']}
)

# Создать словарь преобразования шкалы
scale_mapper = {
    'низкая': 1,
    'средняя': 2,
    'высокая': 3
}

# Заменить значения признаков значениями словаря
df['оценка'].replace(scale_mapper)

0    1
1    1
2    2
3    3
4    2
Name: оценка, dtype: int64

In [10]:
авы = 9
выа = 9
авы + выа

18

Нередко существует признак с классами, которые имеют определенную естественную упорядоченность. Известный пример — шкала Лайкерта:
* полностью согласен;
* согласен;
* нейтрален;
* не согласен;
* категорически не согласен.

При кодировании признака дл] использовании в машинном обучении требуется преобразовать порядковые классы в числовые значения, которые поддерживают идею упорядоченности. Наиболее распространенным подходом является создание словаря, который строковой меткекласса ставит в соответствие чило, а затем применяет это соотнесение к признаку.

Важно, чтобы выбор числовых значений основывался на имеющейся апрорной информации о порядковых классах. Например, метка `'высокая'` буквально в три раза больше метки `'низкая'`. Это нормально в обычных случаях, но может нарушиться, если принятые интервалы между классами не равны:

In [13]:
df = pd.DataFrame(
    {'оценка': [
                'низкая',
                'низкая',
                'средняя',
                'высокая',
                'чуть больше средней'
                ]
    }
)

scale_mapper = {
    'низкая': 1,
    'средняя': 2,
    'чуть больше средней': 3,
    'высокая': 4
}

df['оценка'].replace(scale_mapper)

0    1
1    1
2    2
3    4
4    3
Name: оценка, dtype: int64

В данном примере расстояние между `'низкой'` и `'средней'` равно расстоянию между `'средней'` и `'чуть больше средней'`, что почти наверняка не является точным. Лучший подход сотоит в том, чтобы учитывать чиловые значения, сопоставленным классам:

In [14]:
scale_mapper = {
    'низкая': 1,
    'средняя': 2,
    'чуть больше средней': 2.1,
    'высокая': 4
}

df['оценка'].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    4.0
4    2.1
Name: оценка, dtype: float64

### 3. Кодирование словарей признаков
##### Задача
Дан словарь, и требуется его конвертировать в матрицу признаков
##### Решение
Используем класс-векторизатор словаря `DictVectorizer` библиотеки `scikit-learn`

In [15]:
# Импортировать библиотеку
from sklearn.feature_extraction import DictVectorizer

# Создать словарь
data_dict = [
    {'красный': 2, 'синий': 4},
    {'красный': 4, 'синий': 3},
    {'красный': 1, 'желтый': 2},
    {'красный': 2, 'желтый': 2}
]

# Создать векторизатор словаря
dictvectorizer = DictVectorizer(sparse=False)

# Конвертировать словарь в матрицу признаков
dictvectorizer.fit_transform(data_dict)

array([[0., 2., 4.],
       [0., 4., 3.],
       [2., 1., 0.],
       [2., 2., 0.]])

По умолчанию `DictVectorizer` выводит разряженную матрицу, в которой храняться только элементы со значением отличным от $0$. Это может быть очень полезно, когда имеются массивные матрицы, часто встречающиеся в обработке естественного языка, и требуется минимзировать потребности в оперативной памяти. Менять вывод `DictVectorised` с плотной на разряженную матрицу можно с помощью атрибута `sparse=`.

Имена каждого созданного признака можно получить с помощью метода `get_feature_name`:


In [16]:
# Получить имена признаков
dictvectorizer.get_feature_names()

['желтый', 'красный', 'синий']

In [17]:
# Создать фрейм данных из признаков
features = dictvectorizer.fit_transform(data_dict)
features_names = dictvectorizer.get_feature_names()
pd.DataFrame(data=features, columns=features_names)

,желтый,красный,синий
0,0.0,2.0,4.0
1,0.0,4.0,3.0
2,2.0,1.0,0.0
3,2.0,2.0,0.0


Многие алгоритмы машинного обучения ожидают, что данные будут представлены в виде матрицы. Конвертировать словарь можно используя объект класса `DictVectorizer` библиотеки `scilit-learn`.

Подобная ситуация является обычной в процессе обработки естественного языеа. Например, дана коллекция документов, и для каждого документа имеется словарь, содержащий количество вхождений каждого слова в документ. Используя объект класса `DictVectorizer`, можно создать матрицу признаков, где каждый признак – это количество вхождений слова в кажды йдокумент:

In [19]:
# Создать словари частотностей слов для четырех документов
doc_1_word_count = {'красный': 2, 'синий': 4}
doc_2_word_count = {'красный': 4, 'синий': 3}
doc_3_word_count = {'красный': 1, 'желтый': 2}
doc_4_word_count = {'красный': 2, 'желтый': 2}

# Создать список
doc_word_counts = [doc_1_word_count, doc_2_word_count, doc_3_word_count, doc_4_word_count]

# Конвертировать списоксловарей частотностей слов в матрицу признаков
dictvectorizer.fit_transform(doc_word_counts)

array([[0., 2., 4.],
       [0., 4., 3.],
       [2., 1., 0.],
       [2., 2., 0.]])

В данном примере всего три уникальных слова: `'красный'`, `'желтый'`, `'синий'`, поэтому в нашей матрице всего три признака; однако при работе с реальными данными, например с книгой в библиотеке или интернет страницами, результирующая матрица была бы достаточно большой, и тогда потребовалось бы установить парметр разряженности `sparse=True`.
##### Дополнительные материалы
* "Разряженные матрицы SciPy": https://docs.scipy.org/doc/scipy/reference/sparse.html

### 4. Импутация пропущенных значений классов
##### Задача
Дан категориальный признак, содержащий пропущенные значения, которые требуется заменить предсказанными значениями
##### Решение
Хорошим рещением является натренировать клиссификационный алгоритм машинного обучения для предсказания пропущенных значений. Обучно используют классификатор $k$ ближайших соседей (KNN)

In [22]:
# Загрузить библиотеки
from sklearn.neighbors import KNeighborsClassifier

# Создать матрицу признаков с категориальными признаком
X = np.array([
    [0, 2.1, 1.45],
    [1, 1.18, 1.33],
    [0, 1.22, 1.27],
    [1, -0.21, -1.19]
])

# Создать матрицу признаков с отсутствубщими значениями в категориальном признаке
X_with_nan = np.array([
    [np.nan, 0.87, 1.31],
    [np.nan, -0.67, -0.22]
])

# Натренировать ученика KNN
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:,1:], X[:,0])

# Предсказать класс пропущенных значений
imputed_values = trained_model.predict(X_with_nan[:, 1:])

# Соединить столбец предсказанного класса с другими признаками
X_with_imputed = np.hstack((imputed_values.reshape(-1, 1), X_with_nan[:, 1:]))

# Соединить две матрицы признаков
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

Альтернативным решением явлется заполнение пропущеных значений наиболее частными значениями признаков:

In [23]:
# Импортировать библиотеки
from sklearn.preprocessing import Imputer

# Соединить две матрицы
X_complete = np.vstack((X_with_nan, X))
imputer = Imputer(strategy='most_frequent', axis=0)
imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

Когда в катгориальном признаке имееются пропущенные значения лучшее решение – применить алгоритм машинного обучения для предсказания значений пропущенного наблюдения. Это можно сделать, если рассматривать признак с пропущенными значениями как вектор целей, а другие призанаки – как матрицу признаков. Обще приняти оспользовать алгоритм, которы присваивает пропущенному значению медианный класс $k$ ближайших наблюдений.

Другой вариант – пропущенные значения заполнить наиболее часто используемым классом признака. Хотя он менее сложен, чем алгоритм KNN – он гораздо более масштабируем для более крупных данных. В любом случае рекомендуется включить бинарный признак, указывающий на то, какие наблюдения содержат импутированные значения.
##### Дополнительные материалы
* "Преодоление пропущенных значений в классификаторе на основе случайного леса": https://medium.com/airbnb-engineering/overcoming-missing-values-in-a-random-forest-classifier-7b1fc1fc03ba 
* "Исследование применения $k$ ближайших соседей как метода импутации": http://conteudo.icmc.usp.br/pessoas/gbatista/files/his2002.pdf 

### 5. Работа с несбалансированными классами
##### Задача
Дан вектор целей с очень несьалансированными классами
##### Решение
Собрать больше данных.Если возможно – изменить метрические показатели, используемые для оценивания модели. Если это не работает, рассмотреть возможность использования встроенных в модель параметров веса классов (если таковые имеются), делая повышающий или понижающий отбор.

Для демонстрации данного решения, создадим немного данных с несбалансированными весами. Набор данных ирисов Фишера содержит три сбалансированных класса по 50 наблюдений, каждый из которых указывает на вид цветка: *ирис щетинистый (Iris setosa)*, *ирис виргинский (Iris virginica)* и *ирис разноцветный (Iris versicolor)*. Для того чтобы разбалансировать набор данных необходимо удалить 40 из 50 наблюдений *ириса щетинистого*, а затем объединить классы *ириса виргинского* и *ириса разноцветного*. Конечным результатом станет бинарный вектор целей, казывающий на то, является ли наблюдение цветком *ириса щетинистого* или нет. Результатом станут 10 наблюдений *ирис щетинистый* (класс `0`) и 100 наблюдений не *ирис щетинистый* (класс `1`).

In [24]:
# Загрузить библиотеки
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Загрузить данные цветков ириса Фишера
iris = load_iris()

# Создать матрицу признаков
features = iris.data

# Создать вектор целей
target = iris.target

# Удалить первые 40 наблюдений
features = features[40:, :]
target = target[40:]

# Создать бинарный вектор целей, указывающий, является ли класс 0
target = np.where((target == 0), 0, 1)

# Взглянуть на несбалансированный вектор целей
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Многие алгоритмы библиотеки `scikit-learn` предлагают парметр для взвешивания классов во время тренировки, чтобы противодействовать эффекту их разбалансировки. Классификатор на основе случайного леса `RandomForestClassifier` является популярным классификационным алгоритмом и включает парметр `class_weight=`. 

***Вариант 1***

Можно передать аргумент, явно задающий желаемые веса классов:

In [28]:
# Создать веса
weights = {0: .9, 1 : 0.1}

# Создать классификатор на основе случайного леса с весами
RandomForestClassifier(class_weight=weights)
RandomForestClassifier(bootstrap=True, 
                       class_weight={0: .9, 1: 0.1},
                       criterion='gini',
                       max_depth=None,
                       max_features='auto',
                       max_leaf_nodes=None,
                       min_impurity_decrease=0.0,
                       min_impurity_split=None,
                       min_samples_leaf=1,
                       min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       n_estimators=10,
                       n_jobs=1,
                       oob_score=False,
                       random_state=None,
                       verbose=0,
                       warm_start=False
                      )

RandomForestClassifier(bootstrap=True, class_weight={0: 0.9, 1: 0.1},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=1, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

***Вариант 2***

Передать агрумент `class_weight='balanced'`, который автоматически создает веса классов прпорциональные частотам классов:

In [29]:
# Натренировать случайный лес с помощью сбалансированных весов классов
RandomForestClassifier(class_weight='balanced')
RandomForestClassifier(bootstrap=True, 
                       class_weight='balanced',
                       criterion='gini',
                       max_depth=None,
                       max_features='auto',
                       max_leaf_nodes=None,
                       min_impurity_decrease=0.0,
                       min_impurity_split=None,
                       min_samples_leaf=1,
                       min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       n_estimators=10,
                       n_jobs=1,
                       oob_score=False,
                       random_state=None,
                       verbose=0,
                       warm_start=False
                      )

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=1, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

В качестве альтарнативы можно понизить мажоритарный класс или повысить миноритарный. В понижающем отборе делается произвольная выборка без возврата из мажоритарного класса, т.е. класса с большим количеством наблюдений, чтобы создать новой подмножество наблюдений, равное по размеру миноритарному классу. Например, если миноритарный класс имеет 10 наблюдений, то необходимо будет отобрать сучайным образом 10 наблдейни из мажоритарного класса и использовать эти 20 наблюдений в качестве исходных данных.

In [31]:
# Индексы наблюдений для каждого класса
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]

# Количество наблюдений в каждом классе
n_class0 = len(i_class0)
n_class1 = len(i_class1)

# Для каждого наблдения класса 0 сделать случайную выборку из класса 1 без возврата
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

# Соединить вектор целей класса 0 с вектором целей пониженного класса 1
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [32]:
# Соединить матрицу признаков класса 0 с матрицей признаков класса 1
np.vstack((features[i_class0, :], features[i_class1_downsampled, :]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

In [35]:
# Для каждого наблюдения в классе 1 сделать случайную выборку из класса 0 с возвратом
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

# Соединить повышенный вектор целей класса 0 с вектором целей класса 1
np.concatenate((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [36]:
# Соединить повышенную матрицу признаков класса 0 с матрицей признаков класса 1
np.vstack((features[i_class0_upsampled, :], features[i_class1, :]))[0:5]

array([[4.6, 3.2, 1.4, 0.2],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.3, 1.4, 0.2],
       [5.1, 3.8, 1.9, 0.4],
       [5.1, 3.8, 1.9, 0.4]])

На практике не сбалансированные классы встречаются повсеместно – большинство посетителей не нажимают кнопку **Купить**, и многие виды рака, к счатью, – редки. По этой причине обработка несбалансированных классов является в машинном обучении распространенной задачей.

Лучшей стратегией будет являтся сбор большего числа наблюдений, в особенности наблюдений из миноритарного класса. ОДнкао часто – это просто невозможно.

Другая стратегия заключается в использовании модельного оценочного показателя, более подходящего для несбалансированных классов. В качестве метрики качества модели чсто используется точность, но при наличии несбалансированных классов показатель точности моет не подходить. Например, если только 0.5% наблюдений указывают на редкий вид рака, то даже наивная модель предсказывающая, что ни у кого нет рака, будет точной на 99.5%. Очевидно, что это неидеально. Некоторые более эффективные метрики качества – это матрицы ошибок, прецизиозность, полнота, оценки $F_1$ и кривые $ROC$.

Третья стратегия заключается в использовании параметров взвешивания классов, включенных в реализацию некоторых моделей. Это позоляет настроить алгоритм для несбалансированных классов. Многие классификаторы библиотеки `scikit-learn` имеют парметр `class-weight=`, что делает эту стратегию хорошим вариантом.

Четвертая и пятая стратегии взаимосвязаны: понижающий и повышающий отбор. При понижающем отборе создается случайное подмножество мажоритарного класса одинакового размера с миноритарным классом. При повышающем отборе образцы из миноритарного класса необократно отбираются с заменой, чтобы сделать его равного размера с мажоритарным классом. Решение между использованием понижающего или повышающего отбора зависит от контекста, и в целом необходимо пытаться увидеть, который из них дает лучшие результаты.